In [8]:
import sys
sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [9]:
from model.my_diffusers.models import AutoencoderKL_Pretrained
from utils.lora_utils.lora import inject_trainable_lora_extended
from warper.so2_warper import SO2_warper
from diffusers import AutoencoderKL

In [10]:
from typing import Optional, Union, Tuple, List, Callable, Dict
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as nnf
import numpy as np
import shutil
from torch.optim.adam import Adam
from PIL import Image
import json
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchgeometry as tgm
import cv2
from torch.utils.data import Dataset
from torchvision import datasets
import numpy.typing as npt
from jaxtyping import Float
from torch import Tensor
from pathlib import Path
import json
import random
import os
from torch.utils.data import DataLoader
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity
from math import log10, sqrt
import copy
import tyro
from dataclasses import dataclass

In [11]:
def latent2image(latents, return_type='torch', vae=None):
    latents = 1 / 0.18215 * latents
    image = vae.decode(latents)['sample']
    if return_type == 'np':
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
        image = (image * 255).astype(np.uint8)

    if return_type == 'torch':
        image = (image / 2 + 0.5).clamp(0, 1)

    return image

def torch_to_pil(img):
    if img.dim() == 4:
        img = img.squeeze(0)
        
    img = img.detach().cpu().permute(1, 2, 0).numpy()
    img = (img * 255.0).astype(np.uint8)
    img = Image.fromarray(img)
    
    return img

def mse(original, predicted):
    mse = np.mean((original - predicted) ** 2)
    return mse

def mse_psnr(original, predicted):
    mse_val = mse(original, predicted)
    if(mse_val == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse_val))
    return mse_val, psnr

def cos_sim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))

def sub_vec(A, B):
    return np.linalg.norm(A - B) / np.linalg.norm(A)

In [12]:
def weight_reset(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        m.reset_parameters()
    else:
        print(m)

In [13]:
device = 7
pre_trained = AutoencoderKL_Pretrained.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
raw_decoder = copy.deepcopy(pre_trained)
raw_decoder.apply(weight_reset)
raw_decoder.to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


GroupNorm(32, 128, eps=1e-06, affine=True)
GroupNorm(32, 128, eps=1e-06, affine=True)
Dropout(p=0.0, inplace=False)
SiLU()
ResnetBlock2D(
  (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
  (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (nonlinearity): SiLU()
)
GroupNorm(32, 128, eps=1e-06, affine=True)
GroupNorm(32, 128, eps=1e-06, affine=True)
Dropout(p=0.0, inplace=False)
SiLU()
ResnetBlock2D(
  (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
  (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (nonlinearity): SiLU()
)
ModuleList(
  (0): ResnetBlock2D(
    (norm1): Group

GroupNorm(32, 512, eps=1e-06, affine=True)
Dropout(p=0.0, inplace=False)
SiLU()
ResnetBlock2D(
  (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (nonlinearity): SiLU()
)
GroupNorm(32, 512, eps=1e-06, affine=True)
GroupNorm(32, 512, eps=1e-06, affine=True)
Dropout(p=0.0, inplace=False)
SiLU()
ResnetBlock2D(
  (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (nonlinearity): SiLU()
)
GroupNorm(32, 512, eps=1e-06, affine=True)
GroupNorm(32, 512, eps=1e-06, affine=True)
Dropout(p

Upsample2D(
  (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Upsample2D(
    (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)
UpDecoderBlock2D(
  (resnets): ModuleList(
    (0): ResnetBlock2D(
      (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
      (conv1): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (nonlinearity): SiLU()
      (conv_shortcut): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): ResnetBlock2D(
      (norm1): GroupNorm(32, 256, eps=1e-06, affine=True)
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (conv2): 

AutoencoderKL_Pretrained(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
          (1): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kern

In [15]:
from torchinfo import summary

img = Image.open("/home/jh27kim/warp_latent/dataset/afhq/val/cat/image/flickr_cat_000008.jpg")
img.save("./in.png")
np_img = (np.array(img) / 255.0).astype(np.float32)
torch_img = torch.tensor(np_img)
torch_img = torch_img.permute(2, 0, 1).unsqueeze(0).to(device)
print(torch.max(torch_img), torch.min(torch_img))

latent = raw_decoder.encode(torch_img)
print(latent)
img = raw_decoder.decode(latent)
img = img.squeeze(0).permute(1, 2, 0)
img = img.detach().cpu().numpy() * 255.0
img = img.astype(np.uint8)
img = Image.fromarray(img)
img.save("./out3.png")

tensor(1., device='cuda:7') tensor(0., device='cuda:7')
AutoencoderKLOutput(latent_dist=<model.my_diffusers.models.vae.DiagonalGaussianDistribution object at 0x7f671bb42910>)


TypeError: conv2d() received an invalid combination of arguments - got (AutoencoderKLOutput, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!AutoencoderKLOutput!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!AutoencoderKLOutput!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
